In [1]:
from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
import re, string, unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary


nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#from bs4 import BeautifulSoup
#from nltk import word_tokenize, sent_tokenize

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inlin

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\Kornelius\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import data and prepare data

In [2]:
# Load dataset
df = pd.read_excel('C:/Users/Kornelius/Desktop/Data 2/ecbpr/Warin_Sanger_ECB.xlsx') 
df.head()

,Date,Press,qa
0,1998,"Ladies and gentlemen, in line with our stated ...",We have no indications based on the most recen...
1,1998,"Ladies and gentlemen, as in previous months, t...",The answer to the first question is that it is...
2,1998,"Ladies and gentlemen, the Vice-President and I...",We simply felt that the announcement of a rang...
3,1998,"Ladies and gentlemen, the Vice-President and I...","No, that is not a signal that it is supposed t..."
4,1999,"Ladies and gentlemen, the Vice-President and I...",The main problems have not been technical ones...


In [3]:
#df = df.values.tolist()
# Drop a row by condition
# Print out the first rows of papers
#df = df[df.press]
df = df['qa']

In [4]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(df):
    stop_free = " ".join([i for i in df.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

data = [clean(df).split()for df in df]  

In [5]:
# remove characters and numbers
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [6]:
data = data_words
# Create Dictionary
id2word = corpora.Dictionary(data)
print((id2word))

Dictionary(8371 unique tokens: ['absence', 'accept', 'achieve', 'activity', 'add']...)


In [7]:
# filter words that occur in more than 90% of documents
id2word.filter_extremes(no_above = 0.980)
print(id2word)

Dictionary(3462 unique tokens: ['absence', 'accept', 'achieve', 'activity', 'add']...)


In [8]:
# Create Corpus
texts = data
# Term Document Frequency and creating corpus
corpus = [id2word.doc2bow(text) for text in texts]

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.
This is used as the input by the LDA model.

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [9]:
# Build the LSI Model
lsi_model = LsiModel(corpus=corpus, id2word=id2word, num_topics=1, decay=0.5)

lsi_model5 = LsiModel(corpus=corpus, id2word=id2word, num_topics=5, decay=0.5)

lsi_model10 = LsiModel(corpus=corpus, id2word=id2word, num_topics=10, decay=0.5)

lsi_model20 = LsiModel(corpus=corpus, id2word=id2word, num_topics=20, decay=0.5)

lsi_model50 = LsiModel(corpus=corpus, id2word=id2word, num_topics=50, decay=0.5)

lsi_model100 = LsiModel(corpus=corpus, id2word=id2word, num_topics=100, decay=0.5)



In [10]:
# View Topics
pprint(lsi_model.print_topics(-1))

# Print the Keyword in the 5 topics
pprint(lsi_model5.print_topics(-1))
doc_lsi = lsi_model5[corpus]

# Print the Keyword in the 5 topics
pprint(lsi_model10.print_topics(-1))
doc_lsi = lsi_model10[corpus]

# Print the Keyword in the 20 topics
pprint(lsi_model20.print_topics(-1))
doc_lsi = lsi_model20[corpus]

# Print the Keyword in the 20 topics
pprint(lsi_model50.print_topics(-1))
doc_lsi = lsi_model50[corpus]

# Print the Keyword in the 40 topics
pprint(lsi_model100.print_topics(-1))
doc_lsi = lsi_model100[corpus]

[(0,
  '0.214*"price" + 0.201*"market" + 0.167*"stability" + 0.166*"bank" + '
  '0.151*"see" + 0.150*"inflation" + 0.145*"decision" + 0.139*"council" + '
  '0.139*"area" + 0.136*"regard"')]
[(0,
  '0.214*"price" + 0.201*"market" + 0.167*"stability" + 0.166*"bank" + '
  '0.151*"see" + 0.150*"inflation" + 0.145*"decision" + 0.139*"council" + '
  '0.139*"area" + 0.136*"regard"'),
 (1,
  '0.235*"regard" + -0.201*"bank" + -0.196*"think" + 0.180*"stability" + '
  '-0.152*"actually" + -0.132*"answer" + -0.128*"second" + -0.124*"ecb" + '
  '0.124*"level" + -0.111*"point"'),
 (2,
  '-0.330*"price" + 0.296*"government" + -0.278*"inflation" + 0.266*"decision" '
  '+ 0.204*"country" + 0.170*"measure" + 0.165*"bank" + -0.134*"risk" + '
  '-0.114*"growth" + 0.112*"regard"'),
 (3,
  '0.293*"market" + -0.265*"growth" + 0.201*"bank" + -0.177*"year" + '
  '-0.175*"area" + 0.165*"risk" + -0.141*"country" + -0.139*"development" + '
  '-0.133*"european" + 0.129*"money"'),
 (4,
  '0.417*"inflation" + -0.215

  '-0.129*"money" + 0.120*"european" + -0.119*"central" + -0.116*"guidance" + '
  '0.114*"expectation" + -0.112*"bond"'),
 (31,
  '-0.265*"situation" + -0.202*"growth" + 0.162*"course" + 0.122*"medium" + '
  '0.121*"go" + -0.120*"effect" + 0.117*"last" + 0.107*"analysis" + '
  '0.106*"important" + -0.105*"message"'),
 (32,
  '0.194*"meeting" + 0.193*"exchange" + -0.181*"decision" + 0.148*"look" + '
  '0.145*"regard" + -0.143*"european" + 0.139*"level" + -0.117*"financial" + '
  '0.114*"government" + -0.108*"point"'),
 (33,
  '0.163*"think" + -0.151*"information" + -0.141*"mandate" + -0.128*"area" + '
  '-0.128*"second" + 0.124*"important" + 0.123*"problem" + -0.120*"always" + '
  '0.119*"liquidity" + -0.115*"know"'),
 (34,
  '0.183*"already" + 0.150*"second" + -0.141*"basis" + -0.139*"regard" + '
  '0.131*"particular" + -0.118*"recovery" + -0.110*"functioning" + '
  '-0.110*"system" + -0.109*"again" + -0.108*"two"'),
 (35,
  '-0.192*"central" + -0.184*"part" + -0.135*"quarter" + '
  '-

# Evaluation
Topic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.

The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be? You may summarise it either are ‘cars’ or ‘automobiles’.

In [11]:
from gensim.models.coherencemodel import CoherenceModel

# Compute Perplexity not working?
#print('\nPerplexity: ', lsi_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.



coherence_model_lsa = CoherenceModel(model=lsi_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lsa = coherence_model_lsa.get_coherence()
print('\nCoherence Score: ', coherence_lsa)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model5, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score5: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model10, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score10: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model20, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score20: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model50, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score50: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model100, texts=data, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score100: ', coherence_lsi)


coherence_model_lsa = CoherenceModel(model=lsi_model, texts=texts, dictionary=id2word, coherence='u_mass')
coherence_lsa = coherence_model_lsa.get_coherence()
print('\nCoherence Score: ', coherence_lsa)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model5, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score5: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model10, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score10: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model20, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score20: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model50, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score50: ', coherence_lsi)

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model100, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score100: ', coherence_lsi)


Coherence Score:  0.24093092589115214

Coherence Score5:  0.3345726643735759

Coherence Score10:  0.3180457176656705

Coherence Score20:  0.27723088562831033

Coherence Score50:  0.24804700460808596

Coherence Score100:  0.2315210961071055

Coherence Score:  -0.08113157392042962

Coherence Score5:  -0.3269241017249218

Coherence Score10:  -0.36015904956766503

Coherence Score20:  -0.4233686649108236

Coherence Score50:  -0.8199841826751312

Coherence Score100:  -0.7000874437903168
